In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('data/data_format1/user_log_sample.csv')
df = df[np.logical_or(df.action_type==1, df.action_type==2)]
df.sort_values(by=['user_id', 'time_stamp'], inplace=True)

In [66]:
import pickle
import pandas as pd
import numpy as np
class DatasetGen:
    def __init__(self, log_file='data/data_format1/user_log_sample.csv',
                 user_info_file='data/data_format1/user_info_format1.csv',
                 dataset_file='data/tmall_dataset.pkl',
                 appear_time=10,
                 min_length=3):
        df = pd.read_csv(log_file)
        df.drop_duplicates(inplace=True)
        self.df = df[np.logical_or(df.action_type==1, df.action_type==2)]
        self.df.sort_values(by=['user_id', 'time_stamp'], inplace=True)
        # self.item_buy(appear_time)
        self.user_info_df = pd.read_csv(user_info_file)
        self.process_user_id()
        self.reset_id('user_id')
        self.reset_id('item_id')
        self.reset_id('cat_id')
        self.reset_id('seller_id')
        self.reset_id('brand_id')
        self.dataset_file = dataset_file
        self.training_set = []
        self.testing_set = []
        self.generate_data_set(min_length)
        with open(dataset_file, 'wb') as f:
            pickle.dump((self.training_set, 
                         self.testing_set, 
                         self.user_info_df), f)
    
    def item_buy(self, appear_time=10):
        count = self.df.groupby(by=['item_id']).item_id.agg('count')
        items = set(count[count >= appear_time].index)
        self.df = self.df[self.df.item_id.isin(items)]
        
    def process_user_id(self):
        map_dict = {id:i for i, id in enumerate(set(self.df.user_id.values))}
        self.df.user_id = self.df.user_id.apply(lambda x:map_dict[x])
        self.user_info_df = self.user_info_df[self.user_info_df.user_id.isin(map_dict.keys())]
        self.user_info_df['user_id'] = self.user_info_df['user_id'].apply(lambda x: map_dict[x])
        
    def reset_id(self, column_name):
        map_dict = {id:i for i, id in enumerate(set(self.df[column_name].values))}
        self.df[column_name] = self.df[column_name].apply(lambda x:map_dict[x])
        
    def generate_data_set(self, min_length=3):
        for group in self.df.groupby('user_id'):
            if len(group) < min_length or len(set(group.time_stamp.values)) < 2:
                continue
            self.training_set.append(group[:-1])
            self.testing_set.append(group[-1])
DatasetGen()

(0,)
     user_id  item_id  cat_id  seller_id  brand_id  time_stamp  action_type
656        0       74      54         16        57        1111            2


C:\Users\limuf\Anaconda3\Lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\limuf\Anaconda3\Lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
user_map = {id:i for i, id in enumerate(set(df.user_id.values))}
tmp_df = df.copy(deep=True)

In [12]:
tmp_df.user_id = tmp_df.user_id.apply(lambda x:user_map[x])


In [49]:
for i, block in tmp_df.groupby('user_id'):
    print(len(block))
    if len(block) > 1:
        print(block.iloc[:-2])
        print(block.iloc[-1])
        break

1
27
     user_id  item_id  cat_id  seller_id  brand_id  time_stamp  action_type
799        1  1107286    1272       4540      1878         515            2
922        1   749997     898       4950      2946         531            2
905        1  1004069     707       4549      8150         625            2
906        1  1004069     707       4549      8150         625            2
742        1   924565    1147       4939      7455         712            2
860        1   980619     563       3930      8473         831            2
831        1   853621    1362       1831      7521         911            2
974        1  1039510    1401        192      5638         925            2
800        1   720483     119       2575      7396        1008            2
865        1   990726     600       2792      2431        1011            2
740        1   384988    1328       2257      3155        1015            2
911        1   524366     970       4137      4139        1025            2
915    

In [21]:
tmp_df.head(10)

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
1438,3,700662,953,4420,6198,523,2
1302,3,741961,898,4950,2946,524,2
1451,3,921856,1577,1663,4874,626,2
1456,3,490168,662,1663,4874,626,2
1458,3,1066192,302,1663,4874,626,2
1308,3,926985,1174,270,397,715,2
1477,3,1021007,1326,3620,6118,815,2
1418,3,1014766,180,2375,2470,828,2
1440,3,720990,812,1849,5577,913,2
1435,3,452958,1109,2792,2431,925,2


In [33]:
count = tmp_df.groupby(by=['item_id']).item_id.agg('count')
count.head(10)

item_id
23858     1
40007     1
60221     1
74158     1
81766     2
89484     2
107194    1
121940    1
137298    1
141051    1
Name: item_id, dtype: int64

In [38]:
set(count[count>1].index)



{81766,
 89484,
 251929,
 259853,
 267067,
 316514,
 351303,
 484676,
 585039,
 867263,
 1004069}